In [6]:
from pathlib import Path
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tensorboardX import SummaryWriter
import csv
from mamba_ssm import Mamba

from draw_util import draw_util
import matplotlib.pyplot as plt

from common import load_csv
from pycamera import triangulate, CameraParam

from typing import Dict, List, Tuple
from omegaconf import OmegaConf
%matplotlib widget
%load_ext autoreload
%autoreload 2
CONFIG = OmegaConf.load('../../conf/config.yaml')
MODEL_CONFIG = CONFIG.model[CONFIG.model_name]
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# toggles
VIEW_TRIANGULATIONS = False




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def get_camera_param_dict() -> Dict[str, CameraParam]:
    '''
        Get camera parameters from config.yaml
    '''
    camera_param_dict = {camera_id: CameraParam.from_yaml(Path(CONFIG.camera.folder) / f'{camera_id}_calibration.yaml') for camera_id in CONFIG.camera.cam_ids}
    return camera_param_dict

def test_camera_torch():
    camera_param_dict = get_camera_param_dict()
    p = np.random.rand(3)*5.0
    p = p.tolist()
    cm = list(camera_param_dict.values())[0]
    print('numpy result',cm.proj2img(np.array(p)))
    cm.to_torch()
    print('torch result', cm.proj2img(torch.tensor(p, dtype=torch.float32)))

test_camera_torch()

class MyDataset(Dataset):
    '''
        Dataset, use numpy array for the convinience of triangulation
    '''
    def __init__(self, csv_file):
        self.data = load_csv(csv_file)
    
    def __len__(self):
        return int(self.data[-1, 0]) + 1 
    
    def __getitem__(self, idx):
        return self.data[idx == self.data[:, 0].astype(int),:]


numpy result [ 103.28804977 -551.86928636]
torch result tensor([ 103.2881, -551.8693])


In [8]:
from lfg.util import compute_stamped_triangulations
    
def view_triangulations():
    '''
        Visualize the 3D positions
    '''
    camera_param_dict = {camera_id: CameraParam.from_yaml(Path(CONFIG.camera.folder) / f'{camera_id}_calibration.yaml') for camera_id in CONFIG.camera.cam_ids}
    dataset = MyDataset(Path(CONFIG.dataset.folder) / CONFIG.dataset.camera_data)

    data = dataset[0]
    stamped_positions = compute_stamped_triangulations(data, camera_param_dict)
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(stamped_positions[:,1], stamped_positions[:,2], stamped_positions[:,3], label='3D positions')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.legend()
    for cm in camera_param_dict.values():
        cm.draw(ax,scale=0.20)
    draw_util.set_axes_equal(ax)
    draw_util.set_axes_pane_white(ax)
    draw_util.draw_pinpong_table_outline(ax)

if VIEW_TRIANGULATIONS:
    view_triangulations()

In [9]:
def get_data_loaders() -> Tuple[DataLoader, DataLoader]:
    '''
        Get the data loaders for training and validation
    '''
    dataset = MyDataset(Path(CONFIG.dataset.folder) / CONFIG.dataset.camera_data)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, \
                                    [int(len(dataset)*CONFIG.training_data_split), len(dataset) - int(len(dataset)*CONFIG.training_data_split)])
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    return train_loader, test_loader

def get_summary_writer() -> SummaryWriter:
    '''
        Get the summary writer
    '''
    logdir = Path(MODEL_CONFIG.logdir) / CONFIG.model_name / CONFIG.loss_fn_type
    
    if not logdir.exists():
        logdir.mkdir(parents=True)
        tb_writer = SummaryWriter(logdir=logdir / 'run0')
    else:
        # get the largest number of run in the logdir using pathlib
        paths = list(logdir.glob('run*'))
        indices = [int(str(p).split('run')[-1]) for p in paths]
        if len(indices) == 0:
            max_run_num = 0
        else:
            max_run_num = max(indices)
        tb_writer = SummaryWriter(logdir=logdir / f'run{1+max_run_num}')
    return tb_writer


def get_model() -> nn.Module:
    '''
        Get the model
    '''
    from functools import partial

    if CONFIG.model_name == 'gru':
        from lfg.model import GRUCellModel, gru_pass, gru_autoregr
        model = GRUCellModel(MODEL_CONFIG.input_size, MODEL_CONFIG.hidden_size, MODEL_CONFIG.output_size).to(DEVICE)
        model_pass = gru_pass
        model_autoregr = gru_autoregr
    elif CONFIG.model_name == 'mamba':
        NotImplementedError

        
    return model, model_pass, model_autoregr

In [10]:
from lfg.util import get_uv_from_3d
import omegaconf
from functools import partial
def train_loss(model, model_pass, data, camera_param_dict, criterion):
    data = data[0] # ignore the batch size

    uv_pred = model_pass(model, data, camera_param_dict)
    uv_gt = data[1:, 4:6].float().to(DEVICE)

    if isinstance(CONFIG.camera.image_dim, omegaconf.listconfig.ListConfig):
        normalize = torch.tensor(CONFIG.camera.image_dim, dtype=torch.float32, device=DEVICE)[None,:]
        uv_gt = uv_gt / normalize
        uv_pred = uv_pred / normalize

    loss = criterion(uv_gt, uv_pred)

    return loss

def test_loss(model, model_pass, test_loader,camera_param_dict,criterion):
    '''
        Validate the GRU model
    '''
    with torch.no_grad():
        test_loss = 0.0
        for data in test_loader:
            loss = train_loss(model, model_pass, data, camera_param_dict, criterion)
            test_loss += loss.item()

    return test_loss / len(test_loader)
        
        

def test_autoregr_loss(model, model_autoregr, test_loader, camera_param_dict, criterion):
    '''
        Validate the GRU model
    '''
    with torch.no_grad():
        test_loss = 0.0
        for data in test_loader:
            loss = train_loss(model, partial(model_autoregr,fraction_est=CONFIG.estimation_fraction), data, camera_param_dict, criterion)
            test_loss += loss.item()
        # print(f'Autoregress Loss: {test_loss / len(test_loader)}')
        # tb_writer.add_scalars(main_tag='loss', tag_scalar_dict={'autoregressive':test_loss / len(test_loader)}, global_step=step_count)
        
    return test_loss / len(test_loader)


def train_loop():
    '''
        Train the LSTM model
    '''
    # Set seed
    torch.manual_seed(CONFIG.seed)
    np.random.seed(CONFIG.seed)

    # camera files 
    camera_param_dict = {camera_id: CameraParam.from_yaml(Path(CONFIG.camera.folder) / f'{camera_id}_calibration.yaml') for camera_id in CONFIG.camera.cam_ids}
    
    # Load data
    train_loader, test_loader = get_data_loaders()

    # training loop
    model, model_pass, model_autoregr = get_model()

    if CONFIG.loss_fn_type == 'auto_regressive':
        model_pass_for_loss = partial(model_autoregr, fraction_est=CONFIG.estimation_fraction)
    elif CONFIG.loss_fn_type == 'recurrent':
        model_pass_for_loss = model_pass

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG.lr_init)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=CONFIG.lr_step_size, gamma=CONFIG.lr_gamma)

    # Tensorboard writer
    tb_writer = get_summary_writer()


    step_count = 0
    for epoch in range(CONFIG.num_epochs):
        total_loss = torch.tensor(0.0).to(DEVICE)
        optimizer.zero_grad()
        batch_traj = CONFIG.batch_trajectory
        for i, data in enumerate(train_loader):

            # data (batch_size, seq_len, input_size)
            

            loss = train_loss(model, model_pass_for_loss, data, camera_param_dict, criterion)
            total_loss += loss
            step_count += 1 # for logs

            # batch backward and optimize
            if (i+1) % batch_traj == 0 or i+1 == len(train_loader):
                total_loss /= batch_traj
                # Print loss
                tb_writer.add_scalars(main_tag='loss', tag_scalar_dict={'training':total_loss.item()}, global_step=step_count)
                print(f'Epoch [{epoch+1}/{CONFIG.num_epochs}], Step [{i+1}/{len(train_loader)}], total_loss: {total_loss.item()}')

                # Backward and optimize
                total_loss.backward()
                optimizer.step()
                total_loss = torch.tensor(0.0).to(DEVICE)
                optimizer.zero_grad()
                
        scheduler.step()

        # Test the model every 1 epochs
        if (epoch+1) % 1 == 0:
            validation_loss = test_loss(model, model_pass, test_loader,camera_param_dict,criterion)
            auto_regr_loss = test_autoregr_loss(model, model_autoregr, test_loader, camera_param_dict, criterion)

            print(f'Validation Loss: {validation_loss}, \t Autoregressive Loss: {auto_regr_loss}')
            tb_writer.add_scalars(main_tag='loss', tag_scalar_dict={'testing':validation_loss}, global_step=step_count)
            tb_writer.add_scalars(main_tag='loss', tag_scalar_dict={'auto_regr':auto_regr_loss}, global_step=step_count)
            
        # Save model checkpoint
        torch_save_path = Path(MODEL_CONFIG.model_path) / f'model_{CONFIG.model_name}_{CONFIG.loss_fn_type}.pth'
        os.makedirs(MODEL_CONFIG.model_path, exist_ok=True)
        torch.save(model.state_dict(), torch_save_path)
        print("model saved")

    # Close the writer
    tb_writer.close()

train_loop()

Epoch [1/200], Step [20/400], total_loss: 19576.037109375
Epoch [1/200], Step [40/400], total_loss: 16924.39453125
Epoch [1/200], Step [60/400], total_loss: 19492.25
Epoch [1/200], Step [80/400], total_loss: 17656.0078125
Epoch [1/200], Step [100/400], total_loss: 16965.677734375
Epoch [1/200], Step [120/400], total_loss: 17255.94140625
Epoch [1/200], Step [140/400], total_loss: 18337.421875
Epoch [1/200], Step [160/400], total_loss: 17704.443359375
Epoch [1/200], Step [180/400], total_loss: 18508.919921875
Epoch [1/200], Step [200/400], total_loss: 17225.71484375
Epoch [1/200], Step [220/400], total_loss: 18855.39453125
Epoch [1/200], Step [240/400], total_loss: 17387.904296875
Epoch [1/200], Step [260/400], total_loss: 13282.6083984375
Epoch [1/200], Step [280/400], total_loss: 15182.625
Epoch [1/200], Step [300/400], total_loss: 15377.572265625
Epoch [1/200], Step [320/400], total_loss: 14192.2841796875


In [ ]:
def visualize_predictions():
    # Load data
    train_loader, test_loader = get_data_loaders()
    data = next(iter(test_loader))[0] # ignore the batch size

    # camera files 
    camera_param_dict = {camera_id: CameraParam.from_yaml(Path(CONFIG.camera.folder) / f'{camera_id}_calibration.yaml') for camera_id in CONFIG.camera.cam_ids}
   
    # training loop
    model, model_pass, model_autoregr = get_model()
    state_dict = torch.load( Path(MODEL_CONFIG.model_path) / f'model_{CONFIG.model_name}_{CONFIG.loss_fn_type}.pth')
    model.load_state_dict(state_dict)
    model.eval()


    # predict
    with torch.no_grad():
        uv_pred = model_pass(model, data, camera_param_dict)
        uv_autoregr = model_autoregr(model, data, camera_param_dict, fraction_est=CONFIG.estimation_fraction)
    uv_gt = data[1:, 4:6].float()

    # visualize
    fig = plt.figure(figsize=(12,10))
    ax = fig.subplots(2, 3)
    ax = ax.flatten()

    data = data.cpu().numpy()

    for ax_id, cam_id in enumerate(camera_param_dict.keys()):
        ind = np.where(data[1:, 3].astype(int) == int(cam_id))[0]
        
        ax[ax_id].scatter(uv_pred[ind, 0].cpu().numpy(), uv_pred[ind, 1].cpu().numpy(), color='blue',label='Prediction')
        ax[ax_id].scatter(uv_gt[ind, 0].cpu().numpy(), uv_gt[ind, 1].cpu().numpy(), color='r', label='Ground Truth')
        ax[ax_id].invert_yaxis()
        ax[ax_id].set_title(f'Camera {cam_id}\nRecurrent Prediction')
    
    for ax_id, cam_id in enumerate(camera_param_dict.keys()):
        ind = np.where(data[1:, 3].astype(int) == int(cam_id))[0]
        
        ax[ax_id+3].scatter(uv_autoregr[ind, 0].cpu().numpy(), uv_autoregr[ind, 1].cpu().numpy(), color='blue',label='Prediction')
        ax[ax_id+3].scatter(uv_gt[ind, 0].cpu().numpy(), uv_gt[ind, 1].cpu().numpy(), color='r', label='Ground Truth')
        ax[ax_id+3].invert_yaxis()
        ax[ax_id+3].set_title(f'Camera {cam_id}\nAutoregressive Prediction')
    # for axi in ax:
    #     axi.legend()

visualize_predictions()

